# Simulation Example

Here we demonstrate how to use this package to estimate the multivariate PSD of VAR(2) simulated data.

In [ ]:
! pip install sgvb_psd -q 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sgvb_psd.utils.sim_varma import SimVARMA

np.random.seed(0)

var2 = SimVARMA(
    n_samples=1024,
    var_coeffs=np.array(
        [[[0.5, 0.0], [0.0, -0.3]], [[0.0, 0.0], [0.0, -0.5]]]
    ),
    vma_coeffs=np.array([[[1.0, 0.0], [0.0, 1.0]]]),
    sigma=np.array([[1.0, 0.9], [0.9, 1.0]]),
)
var2

0.50,0.00
0.00,-0.30
0.00,0.00
0.00,-0.50
1.00,0.00
0.00,1.00
1.00,0.90
0.90,1.00


![VAR(2) Data](var2_data.png)

In [5]:
from sgvb_psd.psd_estimator import PSDEstimator

optim = PSDEstimator(
    x=var2.data,
    N_theta=30,
    nchunks=1,
    ntrain_map=1000,
    max_hyperparm_eval=5,
    fs=2 * np.pi,
)
_ = optim.run(lr=0.003);

13:40:41|SGVB-PSD|INFO| Final PSD will be of shape: 512 x 2 x 2
13:40:41|SGVB-PSD|DEBUG| Inputted data shape: (1024, 2)
13:40:41|SGVB-PSD|DEBUG| Model instantiated: SpecPrep(x(t)=(1024, 2), y(f)=(1, 512, 2), Xmat_delta=(512, 32), Xmat_theta=(512, 32))
13:40:41|SGVB-PSD|INFO| Using provided learning rate: 0.003
13:40:41|SGVB-PSD|INFO| Training model
13:40:41|SGVB-PSD|DEBUG| Starting Model Inference Training..
13:40:41|SGVB-PSD|DEBUG| Start Point Estimating... 
13:40:46|SGVB-PSD|DEBUG| MAP Training Time: 4.66s
13:40:46|SGVB-PSD|DEBUG| Start ELBO maximisation... 
13:40:57|SGVB-PSD|DEBUG| VI Time: 10.80s
13:40:57|SGVB-PSD|DEBUG| Total Inference Training Time: 15.57s
13:40:57|SGVB-PSD|INFO| Model trained in 15.84s
13:40:57|SGVB-PSD|INFO| Computing posterior PSDs
13:40:58|SGVB-PSD|INFO| Optimal PSD estimation complete in 1.48s


In [6]:
_ = optim.plot(
    true_psd=[var2.psd, var2.freq],
    off_symlog=False,
    xlims=[0, np.pi],
)
plt.savefig(f"var2_psd.png")
plt.close()

_ = optim.plot_coherence(true_psd=[var2.psd, var2.freq], labels="XY")
plt.savefig(f"var2_coh.png")
plt.close()

_ = optim.plot_vi_losses()
plt.savefig(f"var2_vi_losses.png")
plt.close()


|                       | 
|-----------------------|
| ![VAR(2) PSD][]       |
| ![VAR(2) Coherence][] |
| ![VAR(2) VI Losses][] |

[VAR(2) PSD]: var2_psd.png
[VAR(2) Coherence]: var2_coh.png
[VAR(2) VI Losses]: var2_vi_losses.png

In [15]:
from sgvb_psd.postproc.psd_analyzer import PSDAnalyzer

analyzer = PSDAnalyzer(
    spec_true=var2.psd,
    spectral_density_q=optim.psd_quantiles
)
analyzer

/Users/avaj0001/Documents/projects/sgvb_psd/src/sgvb_psd/postproc/psd_analyzer.py:81: ComplexWarning: Casting complex values to real discards the imaginary part
  N2_VI[i] = np.sum(


,0
L$_2$ Error,0.072419
$\bold{S}_{11}$ CI Length,0.026599
Re($\bold{S}_{12}$) CI Length,0.033488
Im($\bold{S}_{12}$) CI Length,0.023256
$\bold{S}_{22}$ CI Length,0.039866
Coverage,0.748535
